In [1]:
import pandas as pd

# 1. pre-processing

## 加载 ISO

In [2]:
iso = pd.read_csv('Data1/processed/iso.csv',index_col=0).reset_index()
iso.head(3)

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN


## 处理 Population/GDP

In [6]:
ppl_gdp = pd.read_excel('Data1/processed/ppl_gdp.xlsx')
ppl_gdp.head(3)

,CountryName,CountryCode,year,人口,gdp,人均gdp
0,阿鲁巴,ABW,1960,54608.0,NaN,NaN
1,阿鲁巴,ABW,1961,55811.0,NaN,NaN
2,阿鲁巴,ABW,1962,56682.0,NaN,NaN


In [12]:
# 用 map 快速添加英文国家名称列（确保 alpha-3 是唯一键）
eng_name_map = iso.set_index("alpha-3")["name"].to_dict()  # 直接取 "name" 列，不需要双层列表
ppl_gdp["name"] = ppl_gdp["CountryCode"].map(eng_name_map)

# 修正 region 的映射 - 需要先创建一个 alpha-3 到 region 的映射字典
region_map = iso.set_index("alpha-3")["region"].to_dict()  # 创建正确的映射字典
ppl_gdp["region"] = ppl_gdp["CountryCode"].map(region_map)  # 应该用 CountryCode 映射

'''
# 表链接方法（作为替代方案，这是正确的）
ppl_gdp_with_name = pd.merge(
    ppl_gdp,
    iso[["alpha-3", "name", "region"]],  # 可以一次性合并多个需要的列
    left_on="CountryCode",
    right_on="alpha-3",
    how="left"
).drop(columns="alpha-3")  # 合并后可以删除重复的列
ppl_gdp_with_name.head()
'''

# 显示结果
ppl_gdp.head()

,CountryName,CountryCode,year,人口,gdp,人均gdp,name,region
0,阿鲁巴,ABW,1960,54608.0,NaN,NaN,Aruba,Americas
1,阿鲁巴,ABW,1961,55811.0,NaN,NaN,Aruba,Americas
2,阿鲁巴,ABW,1962,56682.0,NaN,NaN,Aruba,Americas
3,阿鲁巴,ABW,1963,57475.0,NaN,NaN,Aruba,Americas
4,阿鲁巴,ABW,1964,58178.0,NaN,NaN,Aruba,Americas


In [13]:
# 删除 eng_name 缺失的行
ppl_gdp_cleaned = ppl_gdp.dropna(subset=["name"])

# 检查剩余数据
print(f"原始数据行数: {len(ppl_gdp)}")
print(f"清理后行数: {len(ppl_gdp_cleaned)}")
print("缺失的国家代码:", ppl_gdp[ppl_gdp["name"].isna()]["CountryCode"].unique())

原始数据行数: 17024
清理后行数: 13760
缺失的国家代码: ['AFE' 'AFW' 'ARB' 'CEB' 'CHI' 'CSS' 'EAP' 'EAR' 'EAS' 'ECA' 'ECS' 'EMU'
 'EUU' 'FCS' 'HIC' 'HPC' 'IBD' 'IBT' 'IDA' 'IDB' 'IDX' 'INX' 'LAC' 'LCN'
 'LDC' 'LIC' 'LMC' 'LMY' 'LTE' 'MEA' 'MIC' 'MNA' 'NAC' 'OED' 'OSS' 'PRE'
 'PSS' 'PST' 'SAS' 'SSA' 'SSF' 'SST' 'TEA' 'TEC' 'TLA' 'TMN' 'TSA' 'TSS'
 'UMC' 'WLD' 'XKX']


In [14]:
gdp_df = ppl_gdp_cleaned.pivot(index="name", columns="year", values="gdp").reset_index()
gdp_df

year,name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.049713e+10,1.913422e+10,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426650e+10,1.450216e+10,NaN
1,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.322815e+10,1.138685e+10,1.186120e+10,1.301973e+10,1.515642e+10,1.540183e+10,1.516273e+10,1.793057e+10,1.891638e+10,2.297768e+10
2,Algeria,2.723615e+09,2.434747e+09,2.001445e+09,2.702982e+09,2.909316e+09,3.136284e+09,3.039859e+09,3.370870e+09,3.852147e+09,...,2.389427e+11,1.874939e+11,1.807638e+11,1.898809e+11,1.945545e+11,1.934597e+11,1.648734e+11,1.862654e+11,2.255603e+11,2.398995e+11
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.430000e+08,6.730000e+08,6.710000e+08,6.120000e+08,6.390000e+08,6.470000e+08,7.210000e+08,7.500000e+08,8.710000e+08,NaN
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.271686e+09,2.789881e+09,2.896610e+09,3.000162e+09,3.218420e+09,3.155149e+09,2.891001e+09,3.324683e+09,3.380602e+09,3.727674e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Virgin Islands (British),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,Virgin Islands (U.S.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.565000e+09,3.663000e+09,3.798000e+09,3.794000e+09,3.923000e+09,4.121000e+09,4.189000e+09,4.444000e+09,NaN,NaN
212,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.322859e+10,4.244449e+10,3.131783e+10,2.684223e+10,2.160616e+10,NaN,NaN,NaN,NaN,NaN
213,Zambia,7.130000e+08,6.962857e+08,6.931429e+08,7.187143e+08,8.394286e+08,1.082857e+09,1.264286e+09,1.368000e+09,1.605857e+09,...,2.714102e+10,2.125122e+10,2.095841e+10,2.587360e+10,2.631151e+10,2.330867e+10,1.813776e+10,2.209642e+10,2.916378e+10,2.816263e+10


In [16]:
ppl_df = ppl_gdp_cleaned.pivot(index="name", columns="year", values="人口").reset_index()
ppl_df

year,name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,9783147.0,10010030.0,10247780.0,...,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0,41128771.0,42239854.0
1,Albania,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,...,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,2837849.0,2811666.0,2777689.0,2745972.0
2,Algeria,11394307.0,11598608.0,11778260.0,11969451.0,12179099.0,12381256.0,12613389.0,12897115.0,13190975.0,...,38760168.0,39543154.0,40339329.0,41136546.0,41927007.0,42705368.0,43451666.0,44177969.0,44903225.0,45606480.0
3,American Samoa,20085.0,20626.0,21272.0,21949.0,22656.0,23391.0,24122.0,24848.0,25608.0,...,52217.0,51368.0,50448.0,49463.0,48424.0,47321.0,46189.0,45035.0,44273.0,43914.0
4,Andorra,9443.0,10216.0,11014.0,11839.0,12690.0,13563.0,14546.0,15745.0,17079.0,...,71621.0,71746.0,72540.0,73837.0,75013.0,76343.0,77700.0,79034.0,79824.0,80088.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Virgin Islands (British),7850.0,7885.0,7902.0,7919.0,7949.0,8018.0,8139.0,8337.0,8649.0,...,28971.0,29366.0,29739.0,30060.0,30335.0,30610.0,30910.0,31122.0,31305.0,31538.0
211,Virgin Islands (U.S.),32500.0,34300.0,35000.0,39800.0,40800.0,43500.0,46200.0,49100.0,55700.0,...,107882.0,107712.0,107516.0,107281.0,107001.0,106669.0,106290.0,105870.0,105413.0,104917.0
212,Yemen,5542459.0,5646668.0,5753386.0,5860197.0,5973803.0,6097298.0,6228430.0,6368014.0,6515904.0,...,27753304.0,28516545.0,29274002.0,30034389.0,30790513.0,31546691.0,32284046.0,32981641.0,33696614.0,34449825.0
213,Zambia,3119430.0,3219451.0,3323427.0,3431381.0,3542764.0,3658024.0,3777680.0,3901288.0,4029173.0,...,15737793.0,16248230.0,16767761.0,17298054.0,17835893.0,18380477.0,18927715.0,19473125.0,20017675.0,20569737.0


In [17]:
population_df = ppl_gdp_cleaned.pivot(index="name", columns="year", values="人口").reset_index()
population_df

year,name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,9783147.0,10010030.0,10247780.0,...,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0,41128771.0,42239854.0
1,Albania,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,1914573.0,1965598.0,2022272.0,...,2889104.0,2880703.0,2876101.0,2873457.0,2866376.0,2854191.0,2837849.0,2811666.0,2777689.0,2745972.0
2,Algeria,11394307.0,11598608.0,11778260.0,11969451.0,12179099.0,12381256.0,12613389.0,12897115.0,13190975.0,...,38760168.0,39543154.0,40339329.0,41136546.0,41927007.0,42705368.0,43451666.0,44177969.0,44903225.0,45606480.0
3,American Samoa,20085.0,20626.0,21272.0,21949.0,22656.0,23391.0,24122.0,24848.0,25608.0,...,52217.0,51368.0,50448.0,49463.0,48424.0,47321.0,46189.0,45035.0,44273.0,43914.0
4,Andorra,9443.0,10216.0,11014.0,11839.0,12690.0,13563.0,14546.0,15745.0,17079.0,...,71621.0,71746.0,72540.0,73837.0,75013.0,76343.0,77700.0,79034.0,79824.0,80088.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,Virgin Islands (British),7850.0,7885.0,7902.0,7919.0,7949.0,8018.0,8139.0,8337.0,8649.0,...,28971.0,29366.0,29739.0,30060.0,30335.0,30610.0,30910.0,31122.0,31305.0,31538.0
211,Virgin Islands (U.S.),32500.0,34300.0,35000.0,39800.0,40800.0,43500.0,46200.0,49100.0,55700.0,...,107882.0,107712.0,107516.0,107281.0,107001.0,106669.0,106290.0,105870.0,105413.0,104917.0
212,Yemen,5542459.0,5646668.0,5753386.0,5860197.0,5973803.0,6097298.0,6228430.0,6368014.0,6515904.0,...,27753304.0,28516545.0,29274002.0,30034389.0,30790513.0,31546691.0,32284046.0,32981641.0,33696614.0,34449825.0
213,Zambia,3119430.0,3219451.0,3323427.0,3431381.0,3542764.0,3658024.0,3777680.0,3901288.0,4029173.0,...,15737793.0,16248230.0,16767761.0,17298054.0,17835893.0,18380477.0,18927715.0,19473125.0,20017675.0,20569737.0


In [18]:
gdp_df.to_csv("Data1/processed/GDP.csv")
population_df.to_csv("Data1/processed/population.csv")

## 处理 Surface

In [3]:
surface = pd.read_csv('BuiltSurfaceCalc/surface_results_rastercalc_first/surface_statistics_iso_summary.csv')
surface.head(3)

,ISO_A3,NAM_0,year,total_surface
0,ABW,Aruba (Neth.),1975,3446472.0
1,ABW,Aruba (Neth.),1980,4812638.0
2,ABW,Aruba (Neth.),1985,6460605.5


In [4]:
# 用 map 快速添加列（需确保 CountryCode 是唯一键）
eng_name_map = iso.set_index("alpha-3")["name"].to_dict()
surface["Countries"] = surface["ISO_A3"].map(eng_name_map)
surface.head()

,ISO_A3,NAM_0,year,total_surface,Countries
0,ABW,Aruba (Neth.),1975,3446472.0,Aruba
1,ABW,Aruba (Neth.),1980,4812638.0,Aruba
2,ABW,Aruba (Neth.),1985,6460605.5,Aruba
3,ABW,Aruba (Neth.),1990,10701824.0,Aruba
4,ABW,Aruba (Neth.),1995,13598802.0,Aruba


In [6]:
surface_df = surface.pivot(index="Countries", columns="year", values="total_surface").reset_index()
surface_df

year,Countries,1975,1980,1985,1990,1995,2000,2005,2010,2015,2020,2025,2030
0,NaN,4.651911e+07,4.903417e+07,5.159184e+07,5.418677e+07,6.056185e+07,6.727144e+07,7.821310e+07,8.986252e+07,1.009455e+08,1.081945e+08,1.140575e+08,1.179649e+08
1,Afghanistan,2.240245e+08,2.499207e+08,2.860036e+08,3.288705e+08,3.564305e+08,3.925731e+08,4.323971e+08,4.883282e+08,5.597124e+08,6.501660e+08,6.974230e+08,7.265302e+08
2,Albania,1.195511e+08,1.285421e+08,1.376679e+08,1.469224e+08,1.655773e+08,1.857004e+08,2.053611e+08,2.259674e+08,2.455197e+08,2.588924e+08,2.671661e+08,2.722590e+08
3,Algeria,1.175932e+09,1.278680e+09,1.383762e+09,1.490991e+09,1.639678e+09,1.796494e+09,1.963683e+09,2.140430e+09,2.312012e+09,2.447871e+09,2.506240e+09,2.547816e+09
4,American Samoa,2.143546e+06,2.584431e+06,3.052325e+06,3.553728e+06,3.709371e+06,3.865298e+06,4.003578e+06,4.145944e+06,4.272875e+06,4.349306e+06,4.349306e+06,4.390132e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,Western Sahara,1.083601e+07,1.138981e+07,1.195084e+07,1.251772e+07,1.396999e+07,1.561607e+07,1.694237e+07,1.840189e+07,1.988653e+07,2.111370e+07,2.143331e+07,2.166018e+07
244,Yemen,1.065825e+08,1.257765e+08,1.522725e+08,1.838727e+08,1.965772e+08,2.118464e+08,2.236755e+08,2.382961e+08,2.571499e+08,2.872784e+08,3.023572e+08,3.132692e+08
245,Zambia,4.463435e+08,5.279606e+08,6.370592e+08,7.608071e+08,8.250948e+08,9.095973e+08,1.015371e+09,1.165021e+09,1.342878e+09,1.515173e+09,1.671511e+09,1.774634e+09
246,Zimbabwe,2.975937e+08,3.403089e+08,3.962399e+08,4.589045e+08,5.048160e+08,5.722144e+08,6.542210e+08,8.004373e+08,1.033890e+09,1.479128e+09,1.760298e+09,1.936121e+09


In [7]:
surface_df.to_csv("Data1/processed/SurfaceArea.csv")

# 2. post processing

### 2.1. lmdi结果分region

In [3]:
lmdi_results_5yrs = pd.read_csv('Data1/results/LMDI_Contributions_5yrs_interval.csv',index_col=0).reset_index()
lmdi_results_5yrs

,Countries,Region,Start_Year,End_Year,ΔTotal,Δtotal_CE/energy_consumption,Δenergy_consumption/GDP,ΔGDP/Population,Δbuilt_surface/Population,Δcement_CS/built_surface
0,Algeria,Africa,1975,1980,-26.735148,-2.599560,-5.277985,71.118306,7.890033,-97.865940
1,Algeria,Africa,1980,1985,1.662086,1.370299,5.265836,10.920952,6.494850,-22.389851
2,Algeria,Africa,1985,1990,-6.010295,-4.113439,8.324942,-5.315408,4.876664,-9.783054
3,Algeria,Africa,1990,1995,-13.021165,-0.190956,24.681683,-31.524017,0.915694,-6.903569
4,Algeria,Africa,1995,2000,-9.595745,0.233208,-16.189702,9.912027,-0.704355,-2.846923
...,...,...,...,...,...,...,...,...,...,...
638,Vietnam,Unknown,1995,2000,1.027820,0.693874,2.129932,6.716315,-3.791381,-4.720921
639,Vietnam,Unknown,2000,2005,-5.821948,1.110699,-1.173749,9.723393,-2.036881,-13.445409
640,Vietnam,Unknown,2005,2010,-1.368196,-0.533061,-7.737999,12.260966,-1.661932,-3.696169
641,Vietnam,Unknown,2010,2015,3.954848,-0.596517,-0.799028,6.500949,-1.402494,0.251938


In [5]:
# 按 Region 和 End_Year 分组求和，保留年份信息
region_year_sum = lmdi_results_5yrs.groupby(['Region','End_Year']).sum().reset_index()

# 计算每个 Region + 年份组合的 ΔTotal 占比（可选）
total_change_by_year = lmdi_results_5yrs.groupby('End_Year')['ΔTotal'].sum().reset_index()
region_year_sum = pd.merge(
    region_year_sum,
    total_change_by_year,
    on=['End_Year'],
    suffixes=('', '_global_total')
)
region_year_sum['ΔTotal_%'] = (region_year_sum['ΔTotal'] / region_year_sum['ΔTotal_global_total']) * 100

# 按 Region 和 End_Year 排序
region_year_sum = region_year_sum.sort_values(['Region', 'End_Year'])
region_year_sum['Start_Year'] = region_year_sum['End_Year'] - 5

# 保存结果
region_year_sum.to_csv('Data1/results/LMDI_Contributions_by_Region_and_Year_5yrs_interval.csv', index=False)

# 显示部分结果
print(region_year_sum[['Region', 'Start_Year', 'End_Year', 'ΔTotal', 'ΔTotal_%']].head(10))

     Region  Start_Year  End_Year      ΔTotal   ΔTotal_%
0    Africa        1975      1980   30.211236  -3.184535
1    Africa        1980      1985   43.099624   3.890029
2    Africa        1985      1990  -21.916248   7.350267
3    Africa        1990      1995  -34.826449 -28.346911
4    Africa        1995      2000    2.233321  -0.392381
5    Africa        2000      2005  -29.930071   2.790074
6    Africa        2005      2010  -21.046199   6.364946
7    Africa        2010      2015  -21.145038   3.335829
8    Africa        2015      2020    1.031332  -0.115543
9  Americas        1975      1980 -355.789693  37.503426


### 2.1. 因素表分region

In [12]:
factors = pd.read_csv('Data1/results/factors.csv',index_col=0)
factors

,Countries,Year,total_CE,cement_CS,energy_consumption,Population,GDP,cement_production,built_surface,region,total_CE/energy_consumption,energy_consumption/GDP,GDP/Population,built_surface/Population,cement_CS/built_surface,CFP
0,Algeria,1975,15.5,0.154741,0.25,0.157247,0.015558,0.90700,1.175932e+09,Africa,62.000000,16.069004,0.098939,7.478254e+09,1.315898e-10,100.167598
1,Algeria,1980,38.5,0.524291,0.64,0.187394,0.042346,3.99080,1.278680e+09,Africa,60.156250,15.113649,0.225972,6.823492e+09,4.100255e-10,73.432450
2,Algeria,1985,57.6,0.767033,0.94,0.221329,0.057938,5.53270,1.383762e+09,Africa,61.276596,16.224276,0.261773,6.252059e+09,5.543099e-10,75.094537
3,Algeria,1990,65.4,0.946670,1.13,0.255181,0.062049,6.50319,1.490991e+09,Africa,57.876106,18.211558,0.243155,5.842882e+09,6.349269e-10,69.084242
4,Algeria,1995,65.2,1.162976,1.13,0.284780,0.041764,6.20000,1.639678e+09,Africa,57.699115,27.056606,0.146654,5.757695e+09,7.092710e-10,56.063076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Vietnam,2000,47.5,2.336391,0.77,0.790011,0.031173,12.50000,3.036673e+09,NaN,61.688312,24.701245,0.039458,3.843834e+09,7.693917e-10,20.330501
796,Vietnam,2005,87.5,6.030925,1.33,0.831421,0.057633,29.00000,3.596242e+09,NaN,65.789474,23.076954,0.069319,4.325416e+09,1.677008e-09,14.508553
797,Vietnam,2010,122.8,9.345256,1.94,0.874110,0.147201,50.00000,4.264285e+09,NaN,63.298969,13.179258,0.168401,4.878430e+09,2.191518e-09,13.140357
798,Vietnam,2015,181.9,10.640411,2.99,0.921914,0.239258,67.64500,4.937336e+09,NaN,60.836120,12.496970,0.259523,5.355528e+09,2.155092e-09,17.095205


In [21]:

# 按 Region 和 End_Year 分组求和，保留年份信息
factors_region_year_sum = factors.groupby(['region','Year']).sum().reset_index()

# 按 Region 和 End_Year 排序
factors_region_year_sum = factors_region_year_sum.sort_values(['region', 'Year'])

# 保存结果
factors_region_year_sum.to_csv('Data1/results/Factors_by_Region_and_Year_5yrs_interval.csv', index=False)

# 显示部分结果
factors_region_year_sum[['region', 'Year', 'total_CE']].head(10)

,region,Year,total_CE
0,Africa,1975,225.8
1,Africa,1980,304.5
2,Africa,1985,445.4
3,Africa,1990,498.2
4,Africa,1995,537.1
5,Africa,2000,587.8
6,Africa,2005,686.7
7,Africa,2010,802.9
8,Africa,2015,843.9
9,Africa,2020,844.6
